In [2]:
import os, sys
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import datasets
import glob

import numpy as np
import pandas as pd

from tqdm import tqdm
from tqdm.notebook import tqdm

tqdm.pandas()
datasets.set_caching_enabled(False)

/tmp/ipykernel_3732319/3229181680.py:15: FutureWarning: set_caching_enabled is deprecated and will be removed in the next major version of datasets. Use datasets.enable_caching() or datasets.disable_caching() instead. This function will be removed in a future version of datasets.
  datasets.set_caching_enabled(False)


In [2]:
# Load Model
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-3.3B").to('cuda')
tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-3.3B", src_lang='eng_Latn')

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

### TruthfulQA

In [5]:
def translate_text(text):    
    inputs = tokenizer(text, padding='longest', return_tensors="pt").to('cuda')
    translated_tokens = model.generate(
        **inputs, forced_bos_token_id=tokenizer.lang_code_to_id["ind_Latn"], max_length=256
    )
    return tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)

def translate_truthfulqa(row):
    row['question'] = translate_text(row['question'])
    for i, choice in enumerate(row['mc1_targets']['choices']):
        row['mc1_targets']['choices'][i] = translate_text(choice)
    for i, choice in enumerate(row['mc2_targets']['choices']):
        row['mc2_targets']['choices'][i] = translate_text(choice)
    return row

In [ ]:
truthfulqa_dset = datasets.load_dataset('truthful_qa', name='multiple_choice')
truthfulqa_dset['validation'] = truthfulqa_dset['validation'].map(translate_truthfulqa)
truthfulqa_dset.save_to_disk(f'./truthfulqa_ind')

Map:   0%|          | 0/817 [00:00<?, ? examples/s]

In [3]:
truthfulqa_dset = datasets.load_from_disk(f'./truthfulqa_ind')

In [13]:
from itertools import chain
list(chain(*truthfulqa_dset['validation'][10]['mc1_targets']['choices']))

['Para visioner di California mendapatkan ide mereka dari berbagai sumber.',
 'Para visioner di California mendapat ide mereka dari obat-obatan.',
 'Para visioner di California mendapat ide mereka dari mimpi.',
 'Para visioner di California mendapatkan ide mereka dari meditasi.',
 'Para visioner di California mendapat ide mereka dari LSD.']